In [1]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import csv
print("Setup Complete")

Setup Complete


In [2]:
data_csv = pd.read_csv('combined_file.csv')
# data_csv.head()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17924\2323715120.py:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_csv = pd.read_csv('combined_file.csv')


In [ ]:
# get name from description
 
def process_text_1(text):
    pattern = r'CT tu([A-Z ]+?)toi MAT TRAN'
    match = re.search(pattern, text)
    if match:
        return process_name(match.group(1))
    
    pattern = r'@PL@([A-Za-z ]+?)Cty CP'
    match = re.search(pattern, text)
    if match:
        return process_name(match.group(1))
    
    pattern = r'(#\.BO):?([A-Za-z ]+?)(\.Remark|\.)'
    match = re.search(pattern, text)
    if match:
        return process_name(match.group(2))
    return ''

def process_text_2(text):
    pattern = r'[A-Z ]+'
    matchs = re.findall(pattern, text)
    name = ''
    for match in matchs:
        name_new = process_name(match)
        if check_name(name_new) and len(name_new) > len(name):
            name = name_new
    if name != '':
        return name.strip()
    return ''

def process_text_3(text):
    pattern = rf'(#.BO|#_|#.|#|QR -)([A-Z -]+)(quyen gop|dong gop|gop|chuyen tien|chuyen FT#|chuyen khoan|chuyen ung ho|ct |ck|chuyen)'
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return process_name(match.group(2))
    
    pattern = rf'(#.BO|#_|#.|#|QR -)([A-Z -]+)(ung ho|uh|ho tro|chia se)'
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return process_name(match.group(2))
    return ''

def process_text_4(text):
    pattern = rf'(tai [a-zA-z ]+\.)([A-Z -]+)(quyen gop|dong gop|gop|chuyen tien|chuyen FT#|chuyen khoan|chuyen ung ho|ct |ck|chuyen)'
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return process_name(match.group(2))
    
    pattern = rf'(tai [a-zA-z ]+\.)([A-Z -]+)(ung ho|uh|ho tro|chia se|#)'
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return process_name(match.group(2))
    return ''

def process_text_5(text):
    pattern = rf'(%|# #|#-#-|#.)([A-Z -]+)(quyen gop|dong gop|gop|chuyen tien|chuyen FT#|chuyen khoan|chuyen ung ho|ct |ck|chuyen)'
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return process_name(match.group(2))
    
    pattern = rf'(%|# #|#-#-|#.)([A-Z -]+)(ung ho|uh|ho tro|chia se)'
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return process_name(match.group(2))
    return ''

def process_name(name):
    if name == '':
        return ''
    name = re.sub(r'IBFT\s?', '', name, flags=re.IGNORECASE) 
    name = re.sub(r'Vietcomban k|VIETCOMB ANK|VIETCOMBA NK|Vietcombank|MBVCB|dung zalopay|qr | TRANSFER|XX ', '', name, flags=re.IGNORECASE)
    name = re.sub(r'(xin|ung ho|dong gop|quyen gop|gui|ft).*', '', name, flags=re.IGNORECASE)
    name = re.sub(r' (?!(y|Y)$)[a-zA-Z]$', '', name, flags=re.IGNORECASE)
    name = re.sub(r'^(con |be |em )', '', name, flags=re.IGNORECASE)
    
    pattern_1 = r'NGUY EN|NGUYE N|HUYN H|HOAN G|DUON G|NGU YEN|PHA M|TRUO NG'
    if re.search(rf'{pattern_1}', name):
        l = name.split(' ')
        name = l[0] + l[1] + ' '.join(l[2:])
    
    if len(name.split()) > 1:
        return name.strip()
    return ''

def check_name(name):
    if name == '':
        return False
    items = name.split(' ')
    # if len(items) < 2 or len(items) > 6:
    #     return False
    if len(items) < 2:
        return False
    if len(items[0]) < 2:
        return False
    if re.search(r'\d', name):
        return False
    
    pattern_1 = r'chuyen tien|chuyen khoan|chuyen ung ho|chuyen FT|ck|gay ra|cuu tro|chia se'
    pattern_2 = r'|tam long|ung ho|uh|chung tay|cua it|dong bao|gop'
    pattern_3 = r'|yagi|mien bac|bao lu|ibt|tw|tp|rem t|moi nguoi|vcb r|chi kh mai'
    if re.search(rf'{pattern_1}{pattern_2}{pattern_3}', name, re.IGNORECASE):
        return False
    return True

def pre_process_text(text):
    if type(text) != str:
        return ''
    text = re.sub(r'"', '', text)
    text = re.sub(r'-', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\d\d\d+', '#', text)
    return '#. ' + text

def pre_process_name_vietin(name, description):
    if type(name) != str:
        return ''
    
    name = re.sub(r'\s+', ' ', name)
    name = re.sub(r'–|-|VNDA|VND|TGTT|TKTT|TKFI', '', name)
    name = re.sub(r'A/C', '', name)
    name = re.sub(r' (?!(y|Y)$)[a-zA-Z]$', '', name, flags=re.IGNORECASE)
    name = name.strip()
    len_name = len(name.split(' '))

    if len_name <= 1 or re.search(r'-|IBFT|momo|BAN VAN DONG CUU TRO TRUNG UONG|TT DV TAI CHINH SO VNPT|NAPAS|tien mat', name, re.IGNORECASE):
        name = getName_vietin(description)
        if name != '':
            return name
        else:
            return 'unknown'

    return name.strip()

def getName(text):
    name = process_text_1(text)
    if check_name(name):
        return name
    
    name = process_text_3(text)
    if check_name(name):
        return name
    
    name = process_text_4(text)
    if check_name(name):
        return name
    
    name = process_text_2(text)
    if check_name(name):
        return name
    
    return ''

def getName_vietin(text):
    
    name = process_text_5(text)
    if check_name(name):
        return name
    
    name = process_text_2(text)
    if check_name(name):
        return name
    return ''


new_csv_file = open('data_with_name.csv', 'w', newline='', encoding='utf-8')
writer_csv = csv.writer(new_csv_file)
writer_csv.writerow(['BankTo', 'Date', 'Name', 'Gender', 'Amount', 'Description'])

for i in range(len(data_csv)):
    if type(data_csv.loc[i, 'Date']) != str:
        continue
    
    description = pre_process_text(data_csv.loc[i, 'Description'])
    
    if data_csv.loc[i, 'Bank_from'] == 'Vietinbank':
        description = re.sub(r'Chuyen tien den tu NAPAS Noi dung:', '%', description)
        description = re.sub(r'IBFT', '', description)
        name = pre_process_name_vietin(data_csv.loc[i, 'Name'], description)

        if name != '':
            writer_csv.writerow([
                            'VIETIN',
                            data_csv.loc[i, 'Date'], 
                            name.upper(), 
                            'unknown',
                            data_csv.loc[i, 'Amount'], 
                            re.sub(r'\s+', ' ', data_csv.loc[i, 'Description'])])
        else:
            writer_csv.writerow([
                            'VIETIN',
                            data_csv.loc[i, 'Date'], 
                            re.sub(r'\s+', ' ', data_csv.loc[i, 'Name']).upper() if type(data_csv.loc[i, 'Name']) == str else 'unknown',
                            'unknown',
                            data_csv.loc[i, 'Amount'], 
                            re.sub(r'\s+', ' ', data_csv.loc[i, 'Description'])])
        continue

    name = getName(description)
    if name != '':
        writer_csv.writerow([data_csv.loc[i, 'Bank_from'],
                            data_csv.loc[i, 'Date'], 
                            name.upper(),
                            'unknown', 
                            data_csv.loc[i, 'Amount'], 
                            re.sub(r'\s+', ' ', data_csv.loc[i, 'Description'])])
    else:
        writer_csv.writerow([data_csv.loc[i, 'Bank_from'],
                            data_csv.loc[i, 'Date'], 
                            re.sub(r'\s+', ' ', data_csv.loc[i, 'Name']).upper() if type(data_csv.loc[i, 'Name']) == str else 'unknown',
                            'unknown', 
                            data_csv.loc[i, 'Amount'], 
                            re.sub(r'\s+', ' ', data_csv.loc[i, 'Description'])])
        
new_csv_file.close()


In [ ]:
# chuyển amount về dạng số, xóa các dòng có giá trị amount = 0, chuyển date về dạng datetime
data_csv = pd.read_csv('data_with_name.csv')
# Function to remove non-digit characters
def clean_amount(amount):
    if pd.isna(amount):
        return amount
    return re.sub(r'\D', '', amount)

# Apply the function to the 'Amount' column
data_csv['Amount'] = data_csv['Amount'].apply(clean_amount)

# Convert to numeric, coercing errors to NaN
data_csv['Amount'] = pd.to_numeric(data_csv['Amount'], errors='coerce')

# Drop rows with NaN values in the 'Amount' column
data_csv = data_csv.dropna(subset=['Amount'])

# Convert 'Amount' column to integer
data_csv['Amount'] = data_csv['Amount'].astype(int)

# Drop rows where 'Amount' is zero
data_csv = data_csv[data_csv['Amount'] != 0]

data_csv['Date'] = data_csv['Date'].apply(lambda x: x.strip('/')+'/2024')
data_csv['Date'] = pd.to_datetime(data_csv['Date'], format='%d/%m/%Y')

data_csv.to_csv('data_with_name.csv', index=False)


In [10]:
data_csv = pd.read_csv('data_with_name.csv')

In [ ]:
# loại bỏ các tên không hợp lệ
def filter_name(name):
    if type(name) != str:
        return False
    if name == 'unknown':
        return False
    if len(name.split()) < 2 or len(name.split()) > 5:
        return False
    
    pattern_1 = r'chuyen tien lien ngan hang|gd|^gia dinh|fan|anh em|sinh vien|sv|hoc sinh|hs|thpt|thcs|dh|sv|mam non'
    pattern_2 = r'|shop|cua hang|chi kh mai|da phat|Bida Bunny|VO CHONG|GIADINH|NGAN HANG'
    pattern_3 = r'|cong ty|cty|MOT CHUT|TAP THE|FC|MN|DONG BAO|CLB|cong dong|co chut|CLUB|VA'
    pattern_4 = r'|me con|KHACH HANG|CT|vn|viet nam|nguoi|ca sy|ca si'
    if re.search(rf'{pattern_1}{pattern_2}{pattern_3}{pattern_4}', name, re.IGNORECASE):
        return False
    return True

data_with_processed_name = data_csv[data_csv['Name'].apply(filter_name)]
data_with_processed_name.to_csv('data_with_processed_name.csv', index=False)